In [41]:
#Import Libraries

import pandas as pd
import numpy as np
import functions_def as fd

In [42]:
# Loading data from csv
data = pd.read_csv("Traffic_Violations.csv")

In [ ]:
# Data Profiling/Inspection
print(data.head(5))
print(data.info())
print(data.columns)
print(data.describe())
print(data.isnull().sum())

# 43 Columns of types: bool(1), float64(3), object(39)
# Total number of rows: 2070115 entries
# Few columns have more null values
    # Search Conducted            786125
    # Search Disposition         1980574
    # Search Outcome              805416
    # Search Reason              1980574
    # Search Reason For Stop      786428
    # Search Type                1980582
    # Search Arrest Reason       2008149

In [44]:
# General EDA

# 1. Remove Duplicates
data.drop_duplicates(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2067761 entries, 0 to 2070114
Data columns (total 43 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   SeqID                    object 
 1   Date Of Stop             object 
 2   Time Of Stop             object 
 3   Agency                   object 
 4   SubAgency                object 
 5   Description              object 
 6   Location                 object 
 7   Latitude                 float64
 8   Longitude                float64
 9   Accident                 object 
 10  Belts                    object 
 11  Personal Injury          object 
 12  Property Damage          object 
 13  Fatal                    object 
 14  Commercial License       object 
 15  HAZMAT                   object 
 16  Commercial Vehicle       object 
 17  Alcohol                  object 
 18  Work Zone                object 
 19  Search Conducted         object 
 20  Search Disposition       object 
 21  Search Outcom

# After removing duplicates, number of entries = 2067761 entries

In [45]:
# Column 1 - SeqID

# There are repetition of rows with same SeqID but with different description and charge. 
# Steps:
# 1. Change the Description and Charge as type str
# 2. Merge the duplicate rows by concatenating the description and charge column

data['Description'] = data['Description'].astype(str).str.lstrip(',| )]\\').str.strip().str.upper()
data['Charge'] = data['Charge'].astype(str)

# Used group by to group rows based on SeqID column
# Sort as been set to False to improve performance for large dataset, as default is True
# as_index = False - to keep the SeqID as normal column, otherwise it will be treated as index because of groupby
# x in aggregation - series of all descirption/charge values of that group.
# separator.join(list_of_strings) - used to merge the description and column
# pd.unique(x) - removes duplicates

data_clean = (
    data.groupby('SeqID', sort=False, as_index=False)
        .agg({
            'Description': lambda x: ' | '.join(pd.unique(x)),            
            'Charge': lambda x: ', '.join(pd.unique(x))
        })
)

print(data_clean.head())

# Keep first occurrence of other columns
data_unique = data.drop_duplicates(subset='SeqID', keep='first')
# Drop the columns Description and Charge from the original df(data_unique)
data_unique = data_unique.drop(columns=['Description', 'Charge'])

# Merge the two df's on column SeqID
data_unique = data_unique.merge(data_clean[['SeqID', 'Description', 'Charge']],on='SeqID',how='left')

print(data_unique.info())

                                  SeqID  \
0  52282e8c-f2e1-4bb5-8509-2d5e4f8da8ca   
1  b66f253b-af29-4bc4-bb73-93755ca2a779   
2  971ef50f-f138-419f-89e5-5d2cc5d7b75a   
3  969c21c0-a843-4e3c-bce6-9abecd6c12f3   
4  df65bb01-5a6f-4f67-a610-082893b1d4fe   

                                         Description  \
0  OPERATING UNREGISTERED MOTOR VEHICLE ON HIGHWA...   
1  DRIVING TO DRIVE MOTOR VEHICLE ON HIGHWAY WITH...   
2  DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...   
3  FAILURE TO STOP AT STOP SIGN | FAILURE TO CONT...   
4  DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...   

                                              Charge  
0       13-401(b1), 13-401(h), 17-107(a1), 13-411(g)  
1  16-101(a1), 13-409(b), 21-310(a), 21-801(b), 1...  
2                    13-401(h), 13-410(e2), 55*, 56*  
3                    21-707(a), 21-801(b), 16-101(b)  
4                                          13-401(h)  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1178080 entries, 0 to 11

In [ ]:
data_unique['Description'].value_counts().sort_index()

# After cleaning the data and removing duplicate rows with same SeqID(by merging the Description and Charge column)
# Number of entries = 11,78,080 entries

In [46]:
# Column 2 - Date of Stop
# From data profiling - There were no rows without Date Of Stop values, and every date was in US format

today = pd.Timestamp.today().normalize()

# This will convert the column to datetime format, and keep all the valid dates and replace all the invalid dates with NaT
# Syntax - Series.where(cond, other=...) , values where the condition is True are kept, where it fails is replaced by pd.NaT
#

data_unique['Date Of Stop'] = (
    pd.to_datetime(data_unique['Date Of Stop'], errors='coerce', format='mixed')
    .where(
        lambda x: (x <= today) & x.dt.year.between(1990, today.year),
        pd.NaT
    )
)

data_unique.head(2)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Location,Latitude,Longitude,Accident,Belts,...,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation,Description,Charge
0,52282e8c-f2e1-4bb5-8509-2d5e4f8da8ca,2023-05-01,23:11:00,MCP,"3rd District, Silver Spring",BRIGGS CHANEY RD @ COLUMIBA PIKE,0.000000,0.00000,No,No,...,False,WHITE,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(0.0, 0.0)",OPERATING UNREGISTERED MOTOR VEHICLE ON HIGHWA...,"13-401(b1), 13-401(h), 17-107(a1), 13-411(g)"
1,b66f253b-af29-4bc4-bb73-93755ca2a779,2023-08-31,16:41:00,MCP,"6th District, Gaithersburg / Montgomery Village",OAKMONT AVE @ GROVEMONT CIR,39.097965,-77.15301,No,No,...,False,HISPANIC,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.097965, -77.15301)",DRIVING TO DRIVE MOTOR VEHICLE ON HIGHWAY WITH...,"16-101(a1), 13-409(b), 21-310(a), 21-801(b), 1..."


In [47]:
# Column 3 - Time Of Stop
data = data_unique.copy()

# Remove all the inconsistencies in the time of stop column, replace . with :
data['Time Of Stop'] = data['Time Of Stop'].astype('str').str.strip().str.replace('.',':')

# Converting it into Time format
data['Time Of Stop'] = pd.to_datetime(data['Time Of Stop'], format='mixed', errors='coerce').dt.time

# Creating a new column for Timestamp
data['Timestamp'] = data['Date Of Stop'].astype('str') + ' ' + data['Time Of Stop'].astype('str')
data.head(5)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Location,Latitude,Longitude,Accident,Belts,...,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation,Description,Charge,Timestamp
0,52282e8c-f2e1-4bb5-8509-2d5e4f8da8ca,2023-05-01,23:11:00,MCP,"3rd District, Silver Spring",BRIGGS CHANEY RD @ COLUMIBA PIKE,0.000000,0.000000,No,No,...,WHITE,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(0.0, 0.0)",OPERATING UNREGISTERED MOTOR VEHICLE ON HIGHWA...,"13-401(b1), 13-401(h), 17-107(a1), 13-411(g)",2023-05-01 23:11:00
1,b66f253b-af29-4bc4-bb73-93755ca2a779,2023-08-31,16:41:00,MCP,"6th District, Gaithersburg / Montgomery Village",OAKMONT AVE @ GROVEMONT CIR,39.097965,-77.153010,No,No,...,HISPANIC,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.097965, -77.15301)",DRIVING TO DRIVE MOTOR VEHICLE ON HIGHWAY WITH...,"16-101(a1), 13-409(b), 21-310(a), 21-801(b), 1...",2023-08-31 16:41:00
2,971ef50f-f138-419f-89e5-5d2cc5d7b75a,2023-04-30,23:55:00,MCP,"4th District, Wheaton",KEMP MILL ROAD @ ALPERT LANE,39.053038,-77.024637,No,No,...,WHITE,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0530383333333, -77.0246366666667)",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,"13-401(h), 13-410(e2), 55*, 56*",2023-04-30 23:55:00
3,969c21c0-a843-4e3c-bce6-9abecd6c12f3,2023-08-31,08:59:00,MCP,"4th District, Wheaton",KAYSON / ESTELLE,39.012720,-77.090825,No,No,...,WHITE,M,BETHESDA,MD,MD,A - Marked Patrol,"(39.0127196666667, -77.0908246666667)",FAILURE TO STOP AT STOP SIGN | FAILURE TO CONT...,"21-707(a), 21-801(b), 16-101(b)",2023-08-31 08:59:00
4,df65bb01-5a6f-4f67-a610-082893b1d4fe,2023-09-01,00:27:00,MCP,"5th District, Germantown",13031 WISTERIA DR,39.178244,-77.270489,No,No,...,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1782438333333, -77.2704891666667)",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,13-401(h),2023-09-01 00:27:00


In [48]:
# Column 4 - Agency
# The column didnt need any cleaning, It had only one unique column

data['Agency'].value_counts()

Agency
MCP    1178080
Name: count, dtype: int64

In [49]:
# Column 5 - SubAgency 
# Nothing to clean here as well.

data['SubAgency'].value_counts().sort_index()

SubAgency
1st District, Rockville                            141399
2nd District, Bethesda                             207166
3rd District, Silver Spring                        206196
4th District, Wheaton                              224364
5th District, Germantown                           151280
6th District, Gaithersburg / Montgomery Village    144910
Headquarters and Special Operations                102761
S15                                                     1
W15                                                     3
Name: count, dtype: int64

In [50]:
# Column 6 - Description 
# Already handled while working on SeqID


In [51]:
# Column 7 - Location

data['Location'] = data['Location'].astype('str').str.strip().str.replace('@','/')
data.head(2)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Location,Latitude,Longitude,Accident,Belts,...,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation,Description,Charge,Timestamp
0,52282e8c-f2e1-4bb5-8509-2d5e4f8da8ca,2023-05-01,23:11:00,MCP,"3rd District, Silver Spring",BRIGGS CHANEY RD / COLUMIBA PIKE,0.000000,0.00000,No,No,...,WHITE,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(0.0, 0.0)",OPERATING UNREGISTERED MOTOR VEHICLE ON HIGHWA...,"13-401(b1), 13-401(h), 17-107(a1), 13-411(g)",2023-05-01 23:11:00
1,b66f253b-af29-4bc4-bb73-93755ca2a779,2023-08-31,16:41:00,MCP,"6th District, Gaithersburg / Montgomery Village",OAKMONT AVE / GROVEMONT CIR,39.097965,-77.15301,No,No,...,HISPANIC,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.097965, -77.15301)",DRIVING TO DRIVE MOTOR VEHICLE ON HIGHWAY WITH...,"16-101(a1), 13-409(b), 21-310(a), 21-801(b), 1...",2023-08-31 16:41:00


In [ ]:
# Column 8 - Latitude

data['Latitude'] = pd.to_numeric(data['Latitude'], errors='coerce').replace(0.0, np.nan).round(6)
data['Latitude'] = data['Latitude'].where((data['Latitude'] >= 18.9) & (data['Latitude'] <= 71.5), np.nan)
data['Latitude'].tail(10)

1178070    39.022674
1178071    38.977886
1178072    39.073293
1178073    39.040975
1178074    39.157210
1178075    39.170505
1178076    38.961660
1178077    38.994304
1178078    39.188985
1178079    39.034735
Name: Latitude, dtype: float64

In [53]:
# Cloumn 9 - Longitude

data['Longitude'] = pd.to_numeric(data['Longitude'], errors='coerce').replace(0, np.nan).round(6)
data['Longitude'] = data['Longitude'].where((data['Longitude'] >= -179.9) & (data['Longitude'] <= -66.9),np.nan)
data[['Latitude','Longitude']].head(10)

,Latitude,Longitude
0,NaN,NaN
1,39.097965,-77.153010
2,39.053038,-77.024637
3,39.012720,-77.090825
4,39.178244,-77.270489
5,38.991480,-77.097113
6,38.995236,-77.031154
7,NaN,NaN
8,NaN,NaN
9,39.066430,-77.165108


In [ ]:
# Columns 10 - 20

boolean_columns = [
    'Accident', 'Belts', 'Personal Injury', 'Property Damage',
    'Fatal', 'Commercial License', 'HAZMAT', 'Commercial Vehicle',
    'Alcohol','Work Zone', 'Search Conducted'
]

TRUE_SET = {'YES', 'Y', 'TRUE', 'T', '1', 1}
FALSE_SET = {'NO', 'N', 'FALSE', 'F', '0', 0, '', np.nan}

def clean_boolean_columns(df):
    for col in boolean_columns:
        s = df[col].str.strip().str.upper()
        df[col] = np.where(s.isin(TRUE_SET), True, False)
    return df

for column in boolean_columns:
    data[column]= data[column].apply(clean_boolean_values)



data[['Accident', 'Belts', 'Personal Injury', 'Property Damage',
    'Fatal', 'Commercial License', 'HAZMAT', 'Commercial Vehicle',
    'Alcohol','Work Zone', 'Search Conducted']].head()

,Accident,Belts,Personal Injury,Property Damage,Fatal,Commercial License,HAZMAT,Commercial Vehicle,Alcohol,Work Zone,Search Conducted
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False


In [ ]:

# Column 21 and 22 -Search Disposition and Search Outcome

# data['Search Disposition'].value_counts(dropna=False).sort_index()
# data['Search Outcome'].value_counts(dropna=False).sort_index()
# Treat blank as 'NA'

search_columns = ['Search Disposition','Search Outcome']

for column in search_columns:
    data[column] = data[column].fillna('NA').astype(str).str.strip().str.upper().replace('NOTHING', 'NA')

In [57]:
#data['Search Reason'].value_counts(dropna=False).sort_index()

# Column 23 - Search Reason
data['Search Reason'] = data['Search Reason'].fillna('NA').astype('str').str.strip().str.upper()

In [58]:
data['Search Reason'].value_counts(dropna=False).sort_index()


Search Reason
ARREST/TOW                      1
CONSENSUAL                   4615
DUI                             1
EXIGENT CIRCUMSTANCES         133
INCIDENT TO ARREST           8795
K-9                           830
NA                        1154918
OTHER                         272
PLAIN VIEW MARIJUANA            1
PROBABLE CAUSE               8512
PROBABLE CAUSE FOR CDS          2
Name: count, dtype: int64

In [59]:
# Column 24 - Search Reason For Stop
# data['Search Reason For Stop'].value_counts().sort_index()

# Column 24
data['Search Reason For Stop'] = data['Search Reason For Stop'].fillna('NA').astype('str').str.strip().str.rstrip("-")

In [60]:
# Column 25 - Search Type

#data['Search Type'].value_counts(dropna=False)
data['Search Type'] = data['Search Type'].fillna('NA').astype('str').str.strip().str.upper()


In [61]:
# Column 26 - Search Arrest Reason
# data['Search Arrest Reason'].value_counts(dropna=False)
data['Search Arrest Reason'] = (
    data['Search Arrest Reason']
    .str.strip()
    .str.upper()
    .replace('MARIHUANA','MARIJUANA')
    .replace('DRIVING','TRAFFIC')
)

In [ ]:
data['Search Arrest Reason'].value_counts(dropna=False)

In [62]:
# Column 27 - State

# data['State'].value_counts(dropna=False)
# data['State'].unique()
# US States
us_states_codes = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN",
    "IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV",
    "NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN",
    "TX","UT","VT","VA","WA","WV","WI","WY"
]

# US Territories
us_territories_codes = [
    "AS","GU","MP","PR","VI"
]

# Canadian Provinces
canadian_provinces_codes = [
    "AB","BC","MB","NB","NL","NS","ON","PE","QC","SK"
]

# Canadian Territories – 2-letter codes
canadian_territories_codes = [
    "NT","NU","YT"
]

# Combined Set
valid_codes = (
    us_states_codes + us_territories_codes +
    canadian_provinces_codes + canadian_territories_codes
)
data['State'] = data['State'].str.upper().str.strip()
data['State'] = data['State'].where(data['State'].isin(valid_codes),np.nan)

In [63]:
# Column 28 - Vehicle Type

#data['VehicleType'].value_counts(dropna=False).sort_index()
data[['Vehicle Code', 'Vehicle Category']] = data['VehicleType'].astype(str).str.strip().str.split(' - ', expand=True)

data[['VehicleType', 'Vehicle Code', 'Vehicle Category']].head(10)

,VehicleType,Vehicle Code,Vehicle Category
0,02 - Automobile,02,Automobile
1,02 - Automobile,02,Automobile
2,02 - Automobile,02,Automobile
3,19 - Moped,19,Moped
4,02 - Automobile,02,Automobile
5,02 - Automobile,02,Automobile
6,02 - Automobile,02,Automobile
7,02 - Automobile,02,Automobile
8,02 - Automobile,02,Automobile
9,02 - Automobile,02,Automobile


In [64]:
# Column 29 - Year

data['Year'] = pd.to_numeric(data['Year'],errors='coerce').where((data['Year'] >= 1960) & (data['Year'] <= 2025),np.nan )
data['Year'] = data['Year'].astype('Int64') 

In [65]:
data.to_csv("Cleaned_data_final_29.csv",index=False)

In [74]:
# Column 30 - Make

data['Make'] = data['Make'].astype(str).str.strip().str.upper().str.replace(r'[^A-Z]', '', regex=True).replace('', 'OTHER')

In [ ]:
# created a map to replace few invalid makes to valid ones
make_map = {
    'AACURA':'ACURA',
    'ACRUAA': 'ACURA',
    'ACUR':'ACURA',
    'HONDRA': 'HONDA',
    'CHEV': 'CHEVROLET',
    'CHEVY': 'CHEVROLET',
    'CHEVEROLT': 'CHEVROLET',
    'DODG':'DODGE',
    'TOY':'TOYOTA',
    'TOYT': 'TOYOTA',
    'TOYOTA': 'TOYOTA',
    'TOTOTA':'TOYOTA',
    'PRUIS': 'TOYOTA',  
    'HOND': 'HONDA',
    'HONDA': 'HONDA',
    'HUNDAI':'HYUNDAI',
    'HYUDAI':'HYUNDAI',
    'HYUN': 'HYUNDAI',
    'HYUNDIA':'HYUNDAI',
    'HYUNDAI': 'HYUNDAI',
    'MAZD': 'MAZDA',
    'MERCEDEZ':'MERCEDES',
    'MERCEDESBENZ':'MERCEDES',
    'MERZ': 'MERCEDES',
    'MERCEDES': 'MERCEDES',
    'SUBA': 'SUBARU',
    'INFI': 'INFINITI',
    'INFINITI': 'INFINITI',
    'EX':'LEXUS',
    'LEX':'LEXUS',
    'EXUS':'LEXUS',
    'LEXS': 'LEXUS',
    'LEXUS': 'LEXUS',
    'LINC': 'LINCOLN',
    'VOLV': 'VOLVO',
    'VOLVO': 'VOLVO',
    'LNDR': 'LAND ROVER',
    'NISS':'NISSAN',
    'MITS': 'MITSUBISHI',
    'CHRYS':'CHRYSLER',
    'CHRISTLER':'CHRYSLER',
    'MERCES':'MERCEDES',
    'BUW': 'BMW',
    'INTN':'INFINITI',
    'LGCH':'LEXUS',
    'MITSUBIHSHI': 'MITSUBISHI',
    'VOVOL': 'VOLVO',
    'FLEETWOOD': 'CADILLAC', 
    'KWORTH': 'KENWORTH',
    'RELEXUS':'LEXUS',
    'NISSIAN':'NISSAN',
    'NISAN':'NISSAN',
    'NSISAN':'NISSAN',
    'MERCEDIS': 'MERCEDES',
    'VOLKWAGON':'VOLKSWAGEN',
    'VOLK':'VOLKSWAGEN',
    'VOKLKSWAGON': 'VOLKSWAGEN',
    'INFINTI':'INFINITI',
    'PORCHE':'PORSCHE',
    'MASE': 'MASERATI',
    'PETE': 'PETERBILT',
    'MNNI': 'MINI',
    'SUZI': 'SUZUKI',
    'SUZU':'SUZUKI',
    'SCIO': 'SCION',
    'FRHT': 'FREIGHTLINER',
    'ISU': 'ISUZU',
    'ISUZ': 'ISUZU',
    'INTL': 'INTERNATIONAL',
    'JAGU':'JAGUAR',
    'TESL':'TESLA',
    'CHRY':'CHRYSLER',
    'NONE':'OTHER',
    'VOLKSWAGON': 'VOLKSWAGEN',
    'VW': 'VOLKSWAGEN',
    'LEXU': 'LEXUS',
    'MERC': 'MERCEDES',
    'CADI': 'CADILLAC',
    'BUIC': 'BUICK',
    'TOYO': 'TOYOTA',
    'PONT': 'PONTIAC',
    'INFINITY': 'INFINITI',
    'KENW': 'KENWORTH',
    'TOYTOTA': 'TOYOTA',
    'IZUZU': 'ISUZU',
    'HUYN': 'HYUNDAI',
    'SAUTN': 'SATURN',
    'HYUNDAY': 'HYUNDAI',
    'SMRT': 'SMART',
    'MER': 'MERCEDES',
    'LAND': 'LANDROVER',
    'HODDA': 'HONDA',
    'ACURAMDX': 'ACURA',
    'MAAZDA': 'MAZDA',
    'LAMBROGHINI': 'LAMBORGHINI',
    'HARLEY': 'HARLEY-DAVIDSON',
    'ACCURA': 'ACURA',
    'VOLKSBLUE': 'VOLKSWAGEN',
    'KIAV': 'KIA',
    'VALTSWAGON': 'VOLKSWAGEN',
    'CHRYTK': 'CHRYSLER',
    'VOLKWAGEN': 'VOLKSWAGEN',
    'VOLSWAGON': 'VOLKSWAGEN',
    'HNDA': 'HONDA',
    'INTE': 'INFINITI',
    'HYANDAI': 'HYUNDAI',
    'HYN': 'HYUNDAI',
    'TOYOVAL': 'TOYOTA',
    'CRYSLER': 'CHRYSLER',
    'BENZ': 'MERCEDES',
    'MERZBENZ': 'MERCEDES',
    'RANG': 'LANDROVER',
    'RAGEROVER': 'LANDROVER',
    'DOEGE': 'DODGE',
    'CVEV': 'CHEVROLET',
    'HUMM': 'HUMMER',
    'MAZ': 'MAZDA',
    'HYUNDA': 'HYUNDAI',
    'KW': 'KENWORTH',
    'HODD': 'HONDA',
    'SAA': 'SAAB',
    'TOTY': 'TOYOTA',
    'PRRSCHE': 'PORSCHE',
    'THOMASBUUS': 'THOMAS',
    'TAOTAO':'TOYOTA',
    'HINDA':'HONDA',
    'MAZADA':'MAZDA',
    'YAMA':'YAMAHA',
    'NISSANSUV':'NISSAN',
    'HYUNDI':'HYUNDAI',
    'MITZ':'MITSUBISHI',
    'MITSU':'MITSUBISHI',
    'SUBU':'SUBARU',
    'CHEVEROLET':'CHEVROLET',
    'SUBURU':'SUBARU',
    'INFIN':'INFINITI',
    'CHRYSTLER':'CHRYSLER',
    'PTRB':'PETERBILT',
    'CADILAC':'CADILLAC',
    'SMRT': 'SMART',
    'PLYM': 'PLYMOUTH',
    'SUB':'SUBARU',
    'PORS':'PORSCHE',
    'TOYTA':'TOYOTA',
    'VOLKS':'VOLKSWAGEN',
    'TOYOT':'TOYOTA',
    'HYUND':'HYUNDAI',
    'ZUZUKI':'SUZUKI'
    
}
data['Make'] = data['Make'].replace(make_map)

In [ ]:
# More cleaning, keeping only the popular makes, rest are replaced with 'OTHER'
make_counts = data['Make'].value_counts()
popular_makes = make_counts[make_counts > 200].index
data['Make'] = data['Make'].where(data['Make'].isin(popular_makes), 'OTHER')

In [85]:
data.to_csv("Cleaned_data_final_30.csv", index=False)

In [108]:
data = pd.read_csv("Cleaned_data_final_30.csv")

In [ ]:
# Column 31 - Model


data['Model']= data['Model'].astype('str').str.strip().str.upper().str.replace(r'[^A_Z0-9]','',regex=True)
data['Model'] = data['Model'].str.lstrip('0')
data['Model'] = data['Model'].str.strip().replace('',np.nan)

In [110]:
data['Model'].value_counts().sort_index()

Model
1            528
10           655
100          162
1000          73
100000000      1
            ... 
ZAZ4           1
ZZ             4
ZZ1200         1
ZZA           52
ZZA4           1
Name: count, Length: 1376, dtype: int64

In [ ]:
# More cleaning
model_counts = data['Model'].value_counts()
popular_models = model_counts[model_counts > 50].index
data['Model'] = data['Model'].where(data['Model'].isin(popular_models), np.nan)

In [112]:
data['Model'].value_counts().sort_index()

Model
1        528
10       655
100      162
1000      73
128       99
        ... 
Z3        93
Z4       280
Z71       68
ZA      2725
ZZA       52
Name: count, Length: 143, dtype: int64

In [ ]:
# Column 32 - Color
# data['Color'].value_counts().sort_index()

color_map = {
    'BLUE, DARK':'DARK BLUE',
    'BLUE, LIGHT':'LIGHT BLUE',
    'GREEN, DK':'DARK GREEN',
    'GREEN, LGT':'LIGHT GREEN'
}

data['Color']= data['Color'].replace(color_map)


In [117]:
# Column 33 - Violation Type

#data['Violation Type'].value_counts().sort_index()

data['Violation Type'] = data['Violation Type'].astype('str').str.strip().str.upper()

In [ ]:
# Column 34 - Charge
# Handled already

In [122]:
# Column 35 - Article

#data['Article'].value_counts().sort_index()
data['Article'] = data['Article'].astype('str').str.strip().str.upper().replace(['NAN','00'],'OTHER')

In [123]:
data['Article'].value_counts().sort_index()

Article
1A                              7
BR                             31
MARYLAND RULES               8527
OTHER                       45718
TG                              2
TRANSPORTATION ARTICLE    1123795
Name: count, dtype: int64

In [ ]:
# Column 36 - Contributed To Accident 

# Data is already clean
data['Contributed To Accident'].value_counts().sort_index()


Contributed To Accident
False    1155583
True       22497
Name: count, dtype: int64

In [ ]:
# Column 37 - Race
# Data is already clean
data['Race'].value_counts().sort_index()

Race
ASIAN               75410
BLACK              349400
HISPANIC           226563
NATIVE AMERICAN      2111
OTHER               89024
WHITE              435572
Name: count, dtype: int64

In [ ]:
# Column 38 - Gender
data['Gender'].value_counts().sort_index()
data['Gender'] = data['Gender'].astype('str').str.replace('U','UNKNOWN')

In [131]:
# Column 39 - Driver City

data['Driver City'] = data['Driver City'].astype('str').str.strip().str.upper().replace(r'[^A-Z]','',regex=True)
data['Driver City'] = data['Driver City'].replace('','OTHER')
data['Driver City'].value_counts().sort_index()

Driver City
AALEXANDRIA        1
ABBINGDON          1
ABBOTTSTOWN        3
ABELL              4
ABERDEEN         116
                ... 
ZELIENOPLE         1
ZEPHYRHILLS        1
ZION               3
ZIONSVILLE         1
ZSILVERSPRING      2
Name: count, Length: 9026, dtype: int64

In [161]:
# More cleaning
city_counts = data['Driver City'].value_counts()
popular_cities = city_counts[city_counts > 50].index
data['Driver City'] = data['Driver City'].where(data['Driver City'].isin(popular_cities), 'OTHER')
data['Driver City'] = data['Driver City'].astype('str').str.replace(r'^[X+]','',regex=True).replace('','OTHER')

In [ ]:
# Column 40 - Driver State
data['Driver State'] = data['Driver State'].str.upper().str.strip()
data['Driver State'] = data['Driver State'].where(data['Driver State'].isin(valid_codes),np.nan)

In [ ]:
# Column 41 - DL State
data['DL State'] = data['DL State'].str.upper().str.strip()
data['DL State'] = data['DL State'].where(data['DL State'].isin(valid_codes),np.nan)

In [ ]:
# Column 42 - Arrest Type
data['Arrest Type'] = data['Arrest Type'].str.upper().str.strip()

In [218]:
# Column 43 - Geolocation
data['Geolocation'] = list(zip(data['Latitude'], data['Longitude']))

mask = (
    data['Latitude'].notna() &
    data['Longitude'].notna() &
    ~((data['Latitude'] == 0.0) & (data['Longitude'] == 0.0))
)

data.loc[~mask, 'Geolocation'] = np.nan


In [222]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1178080 entries, 0 to 1178079
Data columns (total 46 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   SeqID                    1178080 non-null  object 
 1   Date Of Stop             1178080 non-null  object 
 2   Time Of Stop             1178080 non-null  object 
 3   Agency                   1178080 non-null  object 
 4   SubAgency                1178080 non-null  object 
 5   Location                 1178078 non-null  object 
 6   Latitude                 1098558 non-null  float64
 7   Longitude                1098561 non-null  float64
 8   Accident                 1178080 non-null  bool   
 9   Belts                    1178080 non-null  bool   
 10  Personal Injury          1178080 non-null  bool   
 11  Property Damage          1178080 non-null  bool   
 12  Fatal                    1178080 non-null  bool   
 13  Commercial License       1178080 non-null 